# FF25 CAPM

Market tangency test for the FF25 universe using the Finansiering 1 CAPM/GRS setup.


- Excess returns: $r_i = R_i - R_f$ for the 25 portfolios and $r_M = \text{Mkt-RF}$.
- Run $r_i = \alpha_i + \beta_i r_M + \varepsilon_i$ and stack $\hat{\boldsymbol{\alpha}}$, $\hat{\Sigma}_\varepsilon$.
- Market Sharpe $\hat{\theta}_M = \hat{\mu}_M / \hat{\sigma}_M$ (sample mean / sd with $T-1$ in the variance).
- GRS statistic: $\displaystyle \text{GRS} = \frac{T - N - 1}{N} \cdot \frac{\hat{\boldsymbol{\alpha}}^\top \hat{\Sigma}_\varepsilon^{-1} \hat{\boldsymbol{\alpha}}}{1 + \hat{\theta}_M^2}$ which is $F_{N,\,T-N-1}$ under $H_0: \boldsymbol{\alpha}=0$ (market is tangency).


In [13]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
from scipy import stats

sys.path.insert(0, str(Path('..')))
sys.path.insert(0, str(Path('..') / 'src'))

pd.set_option('display.float_format', '{:.6f}'.format)

DATA_DIR = Path('..') / 'data'
START = '1926-07-01'
ALPHA_SIG = 0.05

from finance_data.metrics import tangency_portfolio

## Data

Monthly FF25 excess returns (value-weighted) and the market excess return (Mkt-RF).

In [14]:
from finance_data.datasets import ensure_french_datasets
from finance_data.french import load_us_research_factors_wide

# Load FF25 excess returns and the U.S. market factor
datasets = ensure_french_datasets(output_dir=DATA_DIR, start=START, refresh=False)
ff25_excess = datasets['excess_25'].copy()
ff25_excess.index = pd.to_datetime(ff25_excess.index).to_period('M').to_timestamp('M')

ff_factors, rf_series = load_us_research_factors_wide(start_date=START)
ff_factors.index = pd.to_datetime(ff_factors.index).to_period('M').to_timestamp('M')
market_excess = ff_factors['Mkt-RF']

# Align sample
dates = ff25_excess.index.intersection(market_excess.index)
r_test = ff25_excess.loc[dates].dropna()
r_mkt = market_excess.loc[dates].rename('Mkt-RF')

T, N = r_test.shape
print(f'Aligned FF25 sample: {T} months ({r_test.index.min().date()} to {r_test.index.max().date()}), N={N}')


Aligned FF25 sample: 1191 months (1926-07-31 to 2025-09-30), N=25


/workspaces/finance-data-download-test/notebooks/../src/finance_data/french.py:58: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  fetched = web.DataReader(dataset, "famafrench", start=pdr_start, end=pdr_end)
/workspaces/finance-data-download-test/notebooks/../src/finance_data/french.py:58: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  fetched = web.DataReader(dataset, "famafrench", start=pdr_start, end=pdr_end)


## GRS helper

Vectorized Finansiering 1 CAPM regression and GRS statistic builder.

In [15]:
def grs_test(test_excess: pd.DataFrame, market_excess: pd.Series, alpha_sig: float = 0.05):
    '''Run the CAPM/GRS tangency test for a set of excess returns.'''
    aligned = test_excess.join(market_excess.rename('r_M'), how='inner').dropna()
    if aligned.empty:
        raise ValueError('No overlapping observations between test assets and market')

    r_m = aligned['r_M']
    r_i = aligned[test_excess.columns]
    T_obs, n_assets = r_i.shape
    if T_obs <= n_assets + 1:
        raise ValueError('Need T > N + 1 observations for the GRS statistic')

    X = np.column_stack([np.ones(T_obs), r_m.values])  # T x 2
    XtX = X.T @ X
    try:
        XtX_inv = np.linalg.inv(XtX)
    except np.linalg.LinAlgError:
        XtX_inv = np.linalg.pinv(XtX)

    # OLS betas for all assets at once (2 x N)
    betas = XtX_inv @ X.T @ r_i.values
    alpha_hat = betas[0, :]
    beta_hat = betas[1, :]

    residuals = r_i.values - X @ betas  # T x N
    sigma_eps = residuals.T @ residuals / float(T_obs - 1)
    try:
        sigma_eps_inv = np.linalg.inv(sigma_eps)
    except np.linalg.LinAlgError:
        sigma_eps_inv = np.linalg.pinv(sigma_eps)

    mu_m = float(r_m.mean())
    sigma_m = float(r_m.std(ddof=1))
    theta_m = mu_m / sigma_m

    c = (T_obs - n_assets - 1.0) / float(n_assets)
    quad = alpha_hat @ sigma_eps_inv @ alpha_hat
    grs = c * quad / (1.0 + theta_m ** 2)

    df1 = n_assets
    df2 = T_obs - n_assets - 1
    p_val = float(stats.f.sf(grs, df1, df2))
    crit_val = float(stats.f.ppf(1.0 - alpha_sig, df1, df2))

    sigma2_i = (residuals ** 2).sum(axis=0) / float(T_obs - X.shape[1])
    alpha_se = np.sqrt(sigma2_i * XtX_inv[0, 0])
    beta_se = np.sqrt(sigma2_i * XtX_inv[1, 1])

    alphas = pd.Series(alpha_hat, index=r_i.columns, name='alpha')
    betas = pd.Series(beta_hat, index=r_i.columns, name='beta')
    alpha_se = pd.Series(alpha_se, index=r_i.columns, name='alpha_se')
    beta_se = pd.Series(beta_se, index=r_i.columns, name='beta_se')
    alpha_t = alphas / alpha_se
    beta_t = betas / beta_se

    coef_table = pd.concat([
        alphas,
        alpha_se,
        alpha_t.rename('t_alpha'),
        betas,
        beta_se,
        beta_t.rename('t_beta'),
    ], axis=1)

    return {
        'GRS': float(grs),
        'p_value': p_val,
        'crit_value': crit_val,
        'df1': df1,
        'df2': df2,
        'theta_M': theta_m,
        'mu_M': mu_m,
        'sigma_M': sigma_m,
        'T': T_obs,
        'N': n_assets,
        'alphas': coef_table,
        'residual_cov': pd.DataFrame(sigma_eps, index=r_i.columns, columns=r_i.columns),
    }


## Run the tangency test

In [16]:
grs_result = grs_test(r_test, r_mkt, alpha_sig=ALPHA_SIG)

decision = 'Reject H0: market is tangency' if grs_result['p_value'] < ALPHA_SIG else 'Fail to reject H0'

summary = pd.DataFrame(
    {
        'GRS_stat': [grs_result['GRS']],
        'p_value': [grs_result['p_value']],
        'crit_F(95%)': [grs_result['crit_value']],
        'df1': [grs_result['df1']],
        'df2': [grs_result['df2']],
        'theta_M': [grs_result['theta_M']],
        'mu_M': [grs_result['mu_M']],
        'sigma_M': [grs_result['sigma_M']],
        'T': [grs_result['T']],
        'N': [grs_result['N']],
        'decision': [decision],
    }
)
summary.T


,0
GRS_stat,3.315471
p_value,0.000000
crit_F(95%),1.515568
df1,25
df2,1165
theta_M,0.130227
mu_M,0.006920
sigma_M,0.053138
T,1191
N,25


## Tangency portfolio (max Sharpe)

Max-Sharpe weights from $\Sigma^{-1}\mu$ normalized to sum to 1 (signs allowed).

In [17]:
tp = tangency_portfolio(r_test)

# Portfolio moments
moment_summary = pd.DataFrame(
    {
        'mean': [tp['mean']],
        'vol': [tp['vol']],
        'sr': [tp['sr']],
        'sr_max': [tp['sr_max']],
        'n_assets': [tp['weights'].shape[0]],
    }
).T
moment_summary

# Weight ranking (positive = long, negative = short)
weights_sorted = tp['weights'].sort_values(ascending=False)
weights_top = weights_sorted.head(10).to_frame('weight')
weights_bottom = weights_sorted.tail(10).to_frame('weight')

print('Top 10 weights:')
display(weights_top)

print('Bottom 10 weights:')
display(weights_bottom)

Top 10 weights:


,weight
BIG LoBM,1.106608
SMALL HiBM,1.057633
ME3 BM2,0.818817
ME4 BM1,0.720458
ME5 BM3,0.710456
ME1 BM4,0.613052
ME4 BM4,0.477488
ME2 BM3,0.413610
ME2 BM5,0.375387
ME3 BM3,0.326611


Bottom 10 weights:


,weight
ME5 BM2,-0.261299
ME1 BM3,-0.370186
ME4 BM3,-0.416274
SMALL LoBM,-0.462531
ME4 BM5,-0.510305
ME3 BM1,-0.592502
ME1 BM2,-0.734893
ME2 BM1,-0.756260
ME4 BM2,-0.938670
ME5 BM4,-1.352839


## Replication regression vs market

Regress the tangency portfolio excess return on the market to see how much can be replicated with only market + cash (alpha, beta, $R^2$, tracking error).

In [18]:
# Tangency excess return series
r_tan = r_test @ tp['weights']
r_tan.name = 'r_tan'

# Simple OLS: r_tan = a + b * r_M + eps
T_obs = r_tan.shape[0]
X = np.column_stack([np.ones(T_obs), r_mkt.values])
XtX = X.T @ X
XtX_inv = np.linalg.inv(XtX)
beta_hat = XtX_inv @ X.T @ r_tan.values
alpha_hat, beta_hat_mkt = beta_hat

resid = r_tan.values - X @ beta_hat
sigma2 = (resid @ resid) / float(T_obs - 2)
se_vec = np.sqrt(np.diag(sigma2 * XtX_inv))
se_alpha, se_beta = se_vec

t_alpha = alpha_hat / se_alpha
t_beta = beta_hat_mkt / se_beta

r2 = 1.0 - (resid @ resid) / float(((r_tan - r_tan.mean()) ** 2).sum())
tracking_err = resid.std(ddof=1)

reg_summary = pd.DataFrame(
    {
        'alpha': [alpha_hat],
        'alpha_se': [se_alpha],
        't_alpha': [t_alpha],
        'beta_mkt': [beta_hat_mkt],
        'beta_se': [se_beta],
        't_beta': [t_beta],
        'R2': [r2],
        'tracking_error': [tracking_err],
    }
).T
reg_summary


,0
alpha,0.018645
alpha_se,0.002035
t_alpha,9.163535
beta_mkt,0.650136
beta_se,0.037985
t_beta,17.115745
R2,0.197678
tracking_error,0.069600


## Alphas and betas

Sample CAPM intercepts and slopes (monthly units); sorted by $t_{\alpha}$ to see the largest deviations from zero.

In [19]:
alpha_table = grs_result['alphas'].copy()
alpha_table['alpha_annualized'] = alpha_table['alpha'] * 12
alpha_table_sorted = alpha_table.sort_values('t_alpha', ascending=False)
alpha_table_sorted


,alpha,alpha_se,t_alpha,beta,beta_se,t_beta,alpha_annualized
SMALL HiBM,0.003985,0.001644,2.424455,1.360248,0.030686,44.327346,0.047821
ME3 BM4,0.001880,0.000890,2.111398,1.170643,0.016624,70.419651,0.022561
ME2 BM5,0.002755,0.001358,2.028356,1.376437,0.025360,54.275980,0.033064
ME4 BM4,0.001618,0.000844,1.917819,1.161668,0.015752,73.748907,0.019418
ME3 BM3,0.001375,0.000735,1.871112,1.119468,0.013714,81.627013,0.016495
ME2 BM4,0.001917,0.001073,1.786857,1.215330,0.020029,60.679200,0.023004
ME1 BM4,0.002507,0.001410,1.777862,1.264487,0.026322,48.039126,0.030080
ME3 BM2,0.001183,0.000705,1.676838,1.131137,0.013166,85.915610,0.014190
ME3 BM5,0.001668,0.001241,1.344219,1.356534,0.023165,58.560582,0.020015
ME2 BM3,0.001307,0.000985,1.326789,1.200478,0.018389,65.282666,0.015683
